In [2]:
import os
os.chdir('/Users/sukhmac/Programming/FunCoding/Research/RLProgramming/GittenBandit')  # Set the correct path to your project root

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from Environment.mab_environment import KArmedBandit
from Environment.test_cases import test3
from Strategies.strategy_interface import StrategyInterface

In [ ]:
class QLearning(Strategy):
    def __init__(self, 
                 num_arms,
                 num_states_per_arm, 
                 init_learning_rate, 
                 discount_factor, 
                 temperature_mode):
        super().__init__("QLearning", self)
        self.k = num_arms  # Number of arms (tasks)
        self.n = num_states_per_arm  # Number of states per task
        
        # Initialize the Q-table: (cur_state, action, restart_state, task)
        self.q_table = np.zeros((self.n, 2, self.n, self.k))  # Actions: 0 = Continue (C), 1 = Restart (R)
        
        self.init_learning_rate = init_learning_rate  # adaptive learning rate based on Barto et al 1991 (Appendix B)
        self.tau = 300
        self.cur_learning_rate = np.full((self.n, 2, self.n, self.k), self.init_learning_rate)  # lr for each q value in the Q table
        self.num_updates_q_table = np.zeros((self.n, 2, self.n, self.k))  # stores how many updates for each q value have been performed
        
        self.discount_factor = discount_factor

        self.temperature_mode = temperature_mode
        if self.temperature_mode == "Boltzmann":
            self.max_temp = 200  # Boltzmann temperature
            self.min_temp = 0.5  # based on Barto et al 1991 (Appendix B)
            self.cur_temp = self.max_temp
            self.beta = 0.992
        elif self.temperature_mode == "epsilon-greedy-gittin":
            self.epsilon_greedy = 0.1  # probability of picking action at random
        
        self.time_steps = time_steps
        self.cur_time_step = 0

        self.gittin_history = np.zeros((self.time_steps, self.k, self.n))  # = q_table(i, 0, i, k) history after every update step
